[View in Colaboratory](https://colab.research.google.com/github/seraffin/FailOmen/blob/master/Mariusz/workingNN.ipynb)

In [0]:
import tensorflow as tf

# Helper libraries
import matplotlib.pyplot as plt
import numpy as np

In [0]:
#Multiple fails in one build on test set

def failsCount():
  fala = 0
  for i, a in enumerate(yTest):
    j = 0
    for k, b in enumerate(a):
      if b < 1.0 : j += 1

    if j > 0 : 
      print (i, '. ', j)
    i = i + 1
    fala += j
  print (fala)

In [0]:
# Evaluation function

def evaluation(additionalPredictions):
  
  lenght = len(yTest)

  failPositions = [[] for y in range(lenght)]

  for i, a in enumerate(yTest):

    for j, b in enumerate(a):
      if b == 0 : failPositions[i].append(j);

  predictionsTemp = predictions.copy()
  predictionPositions = [[] for y in range(lenght)]


  for i, a in enumerate(predictionsTemp):

    if len(failPositions[i]) != 0:
      for j in range(len(failPositions[i]) + additionalPredictions):
        argmin = np.argmin(a)
        predictionPositions[i].append(argmin)
        predictionsTemp[i][argmin] = 1

  predictionHits = [[] for y in range(lenght)]

  for i, a in enumerate(failPositions):
    count = 0
    for j, b in enumerate(a):


      for c in predictionPositions[i]:
  #      predictionHits[i].append(predictions[i][c].copy())
  #      print(predictions[i][c])
  #      print(predictions[i][c])
  #      print (b, c)
        if c == b : count += 1

    if len(failPositions) != 0:
      predictionHits[i].insert(0,count)


  failsCount = 0
  hitsCount = 0
  for i, a in enumerate(yTest):
    j = 0

    for k, b in enumerate(a):
      if b < 1.0 : j += 1

  #  if j > 0 :
  #    print (i, '.', j, predictionHits[i])


    failsCount += j
    hitsCount += predictionHits[i][0]

  print('Percentage of fails predicted', hitsCount / failsCount * 100, '%')

In [41]:
#Distribution of argmins through all the predictions
  
def argminsDistribution():
  
  i = 1
  tab = [0] * 542
  for a in predictions:
    j = 0
    for b in a:
      if b < 1.0 : j = j + 1

    #print (i, '. ', j, np.argmin(a))
    tab[np.argmin(a)] += 1
    i = i + 1

  i = 0
  for a in tab:
    if a > 0 : print ('position', i, '\targmin count', a)
    i += 1

IndentationError: ignored

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
!ls

gdrive	sample_data


In [0]:
import pandas as pd

#Importing dataset
dataset = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/convertDataToML.csv', index_col=False)

In [0]:
x = dataset.iloc[:, 0:23739].values
y = dataset.iloc[:, 23739:24281].values

In [7]:
y.shape

(4813, 542)

In [8]:
failCount = 0
passCount = 0
failBuildCount = 0

for a in y:
  if a[np.argmin(a)] == 0 : failBuildCount += 1
  for b in a:
    if b == 0 : failCount += 1
    if b == 1 : passCount += 1
print (failBuildCount)
print (failCount)
print (passCount)
print (failCount / passCount * 100, '%', sep='')

632
973
2607673
0.037312960635785236%


In [0]:
#from sklearn.preprocessing import LabelEncoder
#labelencoder = LabelEncoder()
#y = labelencoder.fit_transform(y)

In [0]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.2)

In [11]:
import keras
from keras.models import Sequential
from keras.layers import Dense
#12140
model = Sequential()
# Adding the input layer and the first hidden layer
model.add(Dense(12140, activation="sigmoid", input_dim=23739, kernel_initializer="uniform")) # TRY smaller input_dim value or less neurons
# Adding the second hidden layer
model.add(Dense(12140, activation = "sigmoid", kernel_initializer="uniform"))
# Adding the output layer
model.add(Dense(542, activation="sigmoid", kernel_initializer="uniform"))

Using TensorFlow backend.


In [12]:
model.compile(optimizer = tf.train.MomentumOptimizer(learning_rate = 0.01, momentum = 0.4), loss = 'binary_crossentropy', metrics = ['accuracy'])

model.fit(xTrain, yTrain, batch_size = 20, epochs = 15)

Epoch 1/15
3850/3850 [==============================] - 38s 10ms/step - loss: 0.1503 - acc: 0.9629
Epoch 2/15
3850/3850 [==============================] - 30s 8ms/step - loss: 0.0248 - acc: 0.9996
Epoch 3/15
3850/3850 [==============================] - 30s 8ms/step - loss: 0.0152 - acc: 0.9996
Epoch 4/15
3850/3850 [==============================] - 29s 8ms/step - loss: 0.0113 - acc: 0.9996
Epoch 5/15
3850/3850 [==============================] - 29s 8ms/step - loss: 0.0093 - acc: 0.9996
Epoch 6/15
3850/3850 [==============================] - 30s 8ms/step - loss: 0.0080 - acc: 0.9996
Epoch 7/15
3850/3850 [==============================] - 30s 8ms/step - loss: 0.0071 - acc: 0.9996
Epoch 8/15
3850/3850 [==============================] - 29s 8ms/step - loss: 0.0065 - acc: 0.9996
Epoch 9/15
3850/3850 [==============================] - 29s 8ms/step - loss: 0.0060 - acc: 0.9996
Epoch 10/15
3850/3850 [==============================] - 29s 8ms/step - loss: 0.0056 - acc: 0.9996
Epoch 11/15
3850/3

In [13]:
lossTest, accTest = model.evaluate(xTest, yTest)

print('Test accuracy:', accTest)
print('Test loss:', lossTest)


963/963 [==============================] - 1s 1ms/step
Test accuracy: 0.9996206614334883
Test loss: 0.00452159610717158


In [0]:
predictions = model.predict(xTest)

In [15]:
#Distributions of argmins through all the predictions
  
def argminsDistribution(): 
  i = 1
  tab = [0] * 542
  for a in predictions:
    j = 0
    for b in a:
      if b < 1.0 : j = j + 1

    #print (i, '. ', j, np.argmin(a))
    tab[np.argmin(a)] += 1
    i = i + 1

  i = 0
  for a in tab:
    if a > 0 : print ('position', i, '\targmin count', a)
    i += 1

position 71 	argmin count 5
position 88 	argmin count 1
position 200 	argmin count 80
position 260 	argmin count 1
position 462 	argmin count 875
position 492 	argmin count 1


In [0]:
#failsCount()

In [43]:
evaluation(15)

32.82828282828283 %


In [19]:
# Evaluation function Test
position2 = 355

predictionsTemp2 = predictions.copy()
yTestTemp = yTest.copy()

for a in range(1):
  testArgmin = np.argmin(yTestTemp[position2])
  print(testArgmin)
  yTestTemp[position2][testArgmin] = 1

for a in range(1 + 3):
  predArgmin = np.argmin(predictionsTemp2[position2])
  print(predArgmin)
  predictionsTemp2[position2][predArgmin] = 1

print(predictionsTemp2[0][0])
print(predictionsTemp2[10])

0
462
200
71
140
0.9980405
[0.9978648  0.9978809  0.9976053  0.9967528  0.99798584 0.9977344
 0.9975852  0.9975944  0.9980451  0.99788266 0.9980842  0.99775606
 0.99789643 0.99808747 0.9976827  0.99797016 0.9978807  0.9978229
 0.9977452  0.99778926 0.9978119  0.9978409  0.9978358  0.99801946
 0.9977034  0.99811673 0.99775213 0.9978956  0.99769825 0.99791044
 0.99753416 0.99759644 0.9977047  0.9978369  0.9977017  0.9976417
 0.9977781  0.99819934 0.9980262  0.99789846 0.99774134 0.9977679
 0.9979837  0.9979328  0.9975236  0.99781764 0.99792993 0.99812526
 0.997908   0.9976882  0.9978599  0.9978859  0.99795866 0.99798054
 0.997693   0.99792224 0.99806756 0.9977679  0.9976847  0.9972428
 0.9982498  0.9979348  0.99813676 0.99800617 0.9979634  0.9977806
 0.9974722  0.9974522  0.9973978  0.99779606 0.99634844 0.9936807
 0.9956417  0.996675   0.99730635 0.99649495 0.99745136 0.9978186
 0.9951491  0.9977337  0.99793375 0.9973183  0.9978904  0.9976961
 0.9978656  0.9978796  0.9977792  0.99813384